In [1]:
#sample running selenium in instagram

#import drivers and librabries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import urllib2
import json
import pandas as pd, numpy as np
import csv
from lxml import html,etree
from datetime import datetime
from datetime import date
import requests,re
import os,sys
import unicodecsv as csv
import argparse
import urllib3
import itertools
import re


In [2]:

class HWHostelList:
    
    def _auto_url(self, locality,country,date_from,date_to,number_of_guests):
        try:
            result = 'https://www.hostelworld.com/search?search_keywords='+str(locality)+',%20'+str(country)+'&country='+str(country)+'&city='+str(locality)+'&date_from='+str(date_from)+'&date_to='+str(date_to)+'&number_of_guests='+str(number_of_guests)+''
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from HostelWorld')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return result
        
    def get_hwurl(self, url):
        try:
            browser = webdriver.Chrome('//selenium/chromedriver')
            browser.get(url)
            #Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            source = browser.page_source    
        except Exception as e:
            raise e
        else:
            return source
    
    #Parse the list of hostels from loc = location
    def parse_hostel(self, html, loc):
        data = []
        parser = etree.HTML(html)
        hostel_list = parser.xpath('//div[@class="d-flex flex-column"]//div[contains(@class,"property-card-right")]')

        for i in range(len(hostel_list)):
            XPATH_HOSTEL_NAME = './/a[contains(@href,"https://www.hostelworld.com/hosteldetails.php/")]//text()'
            XPATH_HOSTEL_LINK = './/a[contains(@href,"https://www.hostelworld.com/hosteldetails.php/")]//@href'
            XPATH_HOSTEL_RATING = './/div[contains(@class,"rating score orange") and not(contains(@class,"small"))]//text()'
            XPATH_HOSTEL_REVIEWS = './/div[contains(@class, "reviews")]//text()'
            XPATH_HOSTEL_PRICE = './/div[@class="prices-container flex-row"]//span'

            raw_hostel_name = hostel_list[i].xpath(XPATH_HOSTEL_NAME)
            raw_hostel_link = hostel_list[i].xpath(XPATH_HOSTEL_LINK)
            raw_hostel_rating = hostel_list[i].xpath(XPATH_HOSTEL_RATING)
            raw_hostel_reviews = hostel_list[i].xpath(XPATH_HOSTEL_REVIEWS)
            raw_hostel_price = hostel_list[i].xpath(XPATH_HOSTEL_PRICE)

            hostel_name = re.sub('[^a-zA-Z0-9 \-\&\@]','',str(raw_hostel_name)) if raw_hostel_name else None
            hostel_link = str(raw_hostel_link).replace('[\'','').replace('\']','') if raw_hostel_link else None
            hostel_rating = re.sub('[^0-9 \.]','',str(raw_hostel_rating)) if raw_hostel_rating else 0
            hostel_reviews = re.sub('[^0-9]','',str(raw_hostel_reviews)) if raw_hostel_reviews else None

            if len(raw_hostel_price) ==4:
                hostel_private_price = re.sub('[^0-9]','',raw_hostel_price[1].text) 
                hostel_dorm_price = re.sub('[^0-9]','',raw_hostel_price[3].text)
            else:
                if raw_hostel_price[0].text == 'No Privates Available':
                    hostel_private_price = 0 
                    hostel_dorm_price = re.sub('[^0-9]','',raw_hostel_price[2].text)
                else:#No Dorms Available
                    hostel_private_price = re.sub('[^0-9]','',raw_hostel_price[1].text)
                    hostel_dorm_price = 0

            data.append([hostel_name,hostel_link,hostel_rating, hostel_reviews, hostel_private_price, hostel_dorm_price])

        data = pd.DataFrame(data, columns=['hostel_name','hostel_link','hostel_rating', 'hostel_reviews','hostel_private_price(USD)', 'hostel_dorm_price(USD)']) 

        data['locality'] = loc

        hostel_id = []
        for index,row in data.iterrows():
            raw_id = ((str(row['hostel_link']).split("?dateFrom", 1)[0]).split("hosteldetails.php/", 1)[1]).split("/")
            hostel_id.append(raw_id[2])
        data['hostel_id'] = hostel_id

        return data
    
    def write_csv(self, csvfile, data):
        try:
            data.to_csv(csvfile, mode='a', index = False, sep=',', encoding='utf-8', header=False)
        except Exception as e:
            raise e
        
    def read_hostelworld(self, url, loc):
        try:
            csvfile = 'hostelworld_hostel.csv'
            last_page = self.get_pagelength('list', url)
            loop = 1
            while loop < last_page+1:
                if loop > 1:
                    new_url = url+'&page='+str(loop)
                    parse = self.get_hwurl(new_url)
                    data = self.parse_hostel(parse, loc)
                    self.write_csv(csvfile, data)
                else:
                    parse = self.get_hwurl(url)
                    data = self.parse_hostel(parse, loc)
                    self.write_csv(csvfile, data)
                loop+=1
                time.sleep(60)
        except Exception as e:
            raise e
            
    def get_pagelength(self, search, url):
        try:
            driver = webdriver.Chrome('//selenium/chromedriver')
            driver.get(url)
            last_page = driver.find_element_by_xpath('.//div[contains(@class,"pagination-last")]')
            last_page.click() 
        except:
            driver.close()
            return 1
        else:
            if search == 'reviews':
                source = driver.page_source
                parser = html.fromstring(source)
                last = parser.xpath('.//div[contains(@class, "pagination-item pagination-current")]//text()')
                result = re.sub("[^0-9 \n\.]","",str(last))
            else:
                last_page_url = driver.current_url
                result = str(last_page_url).replace(url+'&page=',"")
            driver.close()
            return int(result)

    def get_hostelinfo(self, url):
        try:
            source = self.get_hwurl(url)
            parser = html.fromstring(source)
                            
            raw_description = parser.xpath('.//div[contains(@class, "description-property")]//div[contains(@class, "text")]//text()')
            raw_facilities = parser.xpath('.//div[contains(@class, "small-12 columns collapsible-section")]//li[contains(@class, "facility column small-6 medium-4 large-3")]//text()')
            
            #List hostel is index[0] = name, index[1] = local, index[2] = ID
            hostel = ((url.split("?dateFrom", 1)[0]).split("hosteldetails.php/", 1)[1]).split("/")
                        
            hostel_description = str(raw_description).replace('<br>',"").replace('\\n',"").replace('[',"").replace(']',"").lstrip() if raw_description else None
            hostel_description = re.sub("[^a-zA-Z0-9, \n\.]","",hostel_description).strip() 
            
            hostel_facilities = ''.join(str(raw_facilities)).replace('\\n',"").replace(' ',"") if raw_facilities else None
            hostel_facilities = re.sub("[^a-zA-Z0-9, \n\.]","",hostel_facilities)        
        except Exception as e:
            raise e
        else:
            data = pd.DataFrame(data = [[hostel[2], hostel[0], hostel[1], hostel_description, hostel_facilities, url]]
                                ,columns=['hostel_id','hostel_name', 'hostel_locality', 'hostel_description','hostel_facilities','hostel_url'])
            return data
        
    def get_review(self, url):
        try:
            new_url = url+'#reviews'
            page_length = self.get_pagelength('reviews', new_url)
            driver = webdriver.Chrome('//selenium/chromedriver')
            driver.get(new_url)
            #List hostel is index[0] = name, index[1] = local, index[2] = ID
            hostel = ((url.split("?dateFrom", 1)[0]).split("hosteldetails.php/", 1)[1]).split("/")
            page = 1
            reviews = []

            while page < page_length+1:
                XPATH_REVIEW_LIST = driver.find_elements_by_xpath('//div[contains(@class, "property-review row")]')
                XPATH_USER = driver.find_elements_by_xpath('//div[contains(@class,"pt-3")]//li[contains(@class, "name")]')
                XPATH_REVIEW = driver.find_elements_by_xpath('//div[contains(@class,"pt-3")]//div[contains(@class, "notes")]')
                XPATH_DETAILS = driver.find_elements_by_xpath('//div[contains(@class,"pt-3")]//li[contains(@class, "details")]')
                XPATH_RATING = driver.find_elements_by_xpath('//div[contains(@class,"pt-3")]//div[contains(@class, "score")]')
                XPATH_DATE_OF_STAY = driver.find_elements_by_xpath('//div[contains(@class,"pt-3")]//div[contains(@class, "date")]//span')
                time.sleep(5)

                for i in range(len(XPATH_REVIEW_LIST)):
                    raw_user = XPATH_USER[i].text
                    raw_review = XPATH_REVIEW[i].text
                    raw_user_details = XPATH_DETAILS[i].text
                    raw_rating = XPATH_RATING[i].text
                    raw_date_of_stay = XPATH_DATE_OF_STAY[i].text

                    user = re.sub("[^a-zA-Z0-9, \n\.]","",str(raw_user)) if raw_user else None
                    rating = re.sub("[^0-9 \.]","",str(raw_rating)) if raw_rating else None
                    review = raw_review if raw_review else None
                    
                    user_details = re.sub("[^a-zA-Z0-9,-]", "",str(raw_user_details)).split(",")[0] if raw_user_details else None
                    date_of_stay = re.sub("[^a-zA-Z0-9 ]","",str(raw_date_of_stay)) if raw_date_of_stay else None
                    reviews.append([str(hostel[2])+str(page)+str(i),hostel[2], user, rating, review, user_details, date_of_stay])
                    time.sleep(2)
                
                if page < page_length:
                    try:
                        next_page = driver.find_element_by_xpath('.//div[contains(@class,"pagination-next")]')
                        next_page.click()
                    except:
                        return reviews
                    time.sleep(10)    
                page +=1

            reviews = pd.DataFrame(reviews, columns=['review_id','hostel_id','User','Rating', 'Review', 'User_Location', 'Date_of_Stay'])
        except Exception as e:
            raise e
        else:
            driver.close()
            return reviews
    
    def get_ratings(self, url):
        try:
            source = self.get_hwurl(url)
            parser = html.fromstring(source)
            ratings_list = parser.xpath('.//ul[contains(@class, "row rating-breakdown")]//li')
            #List hostel is index[0] = name, index[1] = local, index[2] = ID
            hostel = ((url.split("?dateFrom", 1)[0]).split("hosteldetails.php/", 1)[1]).split("/")
            
            data = []

            for i in range(len(ratings_list)):
                rating = ratings_list[i].xpath('.//p[contains(@class, "rating-label")]//text()')
                data.append([hostel[2],rating[0], rating[1]])
            data = pd.DataFrame(data, columns=['hostel_id','Description','Ratings'])
        except Exception as e:
            raise e
        else:
            return data
        
    def check_exists_by_xpath(self,url, xpath):
        try:
            driver = webdriver.Chrome('//selenium/chromedriver')
            driver.get(url)
            driver.find_element_by_xpath(xpath)
        except NoSuchElementException:
            return False
        else:
            return True
    

In [3]:

w = HWHostelList()

locality ='Siargao Island'
country = 'Philippines'
date_from = '2020-02-01'
date_to = '2020-02-06'
number_of_guests = '1'

url = w._auto_url(locality,country,date_from,date_to,number_of_guests)
w.read_hostelworld(url, locality)


In [ ]:

w = HWHostelList()

columns=['hostel_name','hostel_link','hostel_rating', 'hostel_reviews','hostel_private_price(USD)','hostel_dorm_price(USD)','Locality','hostel_id']
hostel_list = pd.read_csv("hostelworld_hostel.csv", names=columns, header=None)

for index, row in hostel_list.iterrows():
    url = row['hostel_link']
    
    info = w.get_hostelinfo(url)
    w.write_csv('hostelinfo.csv', info)
    
    if row['hostel_reviews'] > 0:
        ratings = w.get_ratings(url)
        w.write_csv('hostelratings.csv', ratings)
        review = w.get_review(url)
        w.write_csv('hostelreviews.csv', review)
    
    time.sleep(10)
